# A Physicist Plans a Party (Comparing Prices with Monte Carlo Simulation)

> Adam Lyon, January 2018

We're planning our daughter's Bat Mitzvah. Banquet halls offer different catering and beverage plans that we need to evaluate. It's hard to do this without knowing *a priori* who is coming and what everyone will drink. So we have to guess. A Monte Carlo simulation is one way to do the guessing. Instead of trying to guess particular situations, we'll try to guess *all* situations (or at least a large sample). We create many, many Bat Mitzvahs randomly selecting guest count and drink selections. We can then apply different the pricing rules to the parties. That will create distributions of total prices and other results that we can compare to each other. 

In this notebook, we'll examine two banquet halls WT and cpr (names changed out of paranoia). 

Speed is important here as we want to make as many guesses as we can. I'm going to try to use the [Julia Language](https://julialang.org) as it is just-in-time compiled. Exploiting [parallelization](https://docs.julialang.org/en/stable/manual/parallel-computing) is easier in Julia than in other languages. My laptop has four [hyperthreaded](https://en.wikipedia.org/wiki/Hyper-threading) cores, so I can get eight simultaneous threads going. That's a free factor of ~8 in speed! With the current code here I can generate a million Bat Mitvah's in a minute. 

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#Configure-Julia" data-toc-modified-id="Configure-Julia-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Configure Julia</a></span></li><li><span><a href="#Party-Parameters" data-toc-modified-id="Party-Parameters-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Party Parameters</a></span><ul class="toc-item"><li><span><a href="#Define-Guests" data-toc-modified-id="Define-Guests-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Define Guests</a></span></li><li><span><a href="#Define-drinks" data-toc-modified-id="Define-drinks-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Define drinks</a></span></li><li><span><a href="#Define-costs" data-toc-modified-id="Define-costs-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Define costs</a></span></li></ul></li><li><span><a href="#Make-Parties" data-toc-modified-id="Make-Parties-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Make Parties</a></span><ul class="toc-item"><li><span><a href="#Run-single-threaded" data-toc-modified-id="Run-single-threaded-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Run single threaded</a></span></li><li><span><a href="#Restore-from-file" data-toc-modified-id="Restore-from-file-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Restore from file</a></span></li><li><span><a href="#Run-in-Parallel" data-toc-modified-id="Run-in-Parallel-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Run in Parallel</a></span></li></ul></li><li><span><a href="#Results" data-toc-modified-id="Results-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Results</a></span><ul class="toc-item"><li><span><a href="#Check-coverage" data-toc-modified-id="Check-coverage-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Check coverage</a></span></li><li><span><a href="#Distributions-of-guests" data-toc-modified-id="Distributions-of-guests-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Distributions of guests</a></span></li><li><span><a href="#Distributions-of-drinks" data-toc-modified-id="Distributions-of-drinks-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Distributions of drinks</a></span></li><li><span><a href="#Cost-results" data-toc-modified-id="Cost-results-4.4"><span class="toc-item-num">4.4&nbsp;&nbsp;</span>Cost results</a></span></li><li><span><a href="#Explore-cpr-where-by-package-is-favored" data-toc-modified-id="Explore-cpr-where-by-package-is-favored-4.5"><span class="toc-item-num">4.5&nbsp;&nbsp;</span>Explore cpr where by package is favored</a></span></li></ul></li><li><span><a href="#Back-Matter" data-toc-modified-id="Back-Matter-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Back Matter</a></span></li></ul></div>

## Configure Julia

In [1]:
addprocs() ; nworkers() # Use all available threads on the machine

8

In [2]:
# Load packages ... @everywhere loads them on all the worker threads
@everywhere begin
    using Base.Test
    using Lazy: @>, @>>
    using Distributions
    using Plots
    using DataFrames
    using DataFramesMeta
    using StatPlots
end

A note about the `@>` syntax you'll see below as it will likely look unfamiliar. The [Lazy.jl](https://github.com/MikeInnes/Lazy.jl) package has the `@>` and `@>>` (and other) macros to create a pipeline of commands. `@>` threads the result of a function into the first argument position of the next function on the line. `@>>` threads the result of a function into the *last* argument position of the next function (needed rarely). For example,

```julia
mean( @select(x, :nA ) )  # Take the mean of the :nA column of the data frame x
@> x @select(:nA) mean    # This line does the same as above

ndupl = sum( nonunique( @select(x, :nA, :nT, :nC, :nM, :nW, :nSA, :nSK) ) )
ndupl = @> x @select(:nA, :nT, :nC, :nM, :nW, :nSA, :nSK) nonunique sum()  # Same as line above

printfmtln("{1:.2f}%", ndupl/nrow(x)*100 )
@>> ndupl/nrow(x)*100 printfmtln("{1:.2f}%")  # Same as line above
```

I find the threading macros make complex expressions easier to read as you can follow the flow from left to right instead of from inner parentheses to outer. 

In [3]:
srand()   # Seed the global random number generator

MersenneTwister(UInt32[0x77cb6d4b, 0xff2b0920, 0x6d534984, 0x3aa18795], Base.dSFMT.DSFMT_state(Int32[-1661869821, 1073530790, 4846302, 1072914942, -880557277, 1073424325, 1261606242, 1072913035, 1648174384, 1073076698  …  1201781392, 1073499427, -2121823517, 1073707440, 36385352, 2140781794, 235639316, -570282858, 382, 0]), [1.35901, 1.09064, 1.50988, 1.21048, 1.49833, 1.7068, 1.93316, 1.51821, 1.05202, 1.14986  …  1.91641, 1.74119, 1.37645, 1.17618, 1.48309, 1.89624, 1.52158, 1.43765, 1.96226, 1.20451], 382)

*Assumptions and limitations:*

* We are going to ignore correlations between guests. For example, many guests come in families. We're going to ignore that for now.
* We'll assume that no one drinks beer

Party parameters from the latest [guest list](https://docs.google.com/spreadsheets/d/1XnHJmx3Ke0D1UaFNWex0c4MNv9CawyiZPqVUIqkzePY/edit#gid=0)

In [4]:
function printNicely(x) 
    """
    Julia prints unsigned integers in hex by default. Print in decimal instead
    """
    [println(x[i]) for i in 1:length(x)]
    nothing
end

printNicely (generic function with 1 method)

## Party Parameters

Here is where we define the parameters of the parties

### Define Guests

In [6]:
# Ranges of guests
@everywhere begin
    const adultRange  = (41, 71)   # Adult food rate and drinks alcohol
    const teenRange   = (18, 27)   # 13 and over (adult food rate, but no alcohol)
    const childRange  = (11, 22);  # Child fee 
end

In [7]:
@everywhere function randIntInRange(range::Tuple{Int, Int})
    """
    Return a flat random distribution within the given range
    """
    
    floor(Int, rand()*(range[2]-range[1]+1)+range[1])
end

@everywhere function randomsInRanges(ranges...)
    """
    Pull a random number for each range. The output is in the same order of the ranges
    """
    [randIntInRange(ranges[i]) for i in 1:length(ranges)]
end

### Define drinks

We need to determine how many drinks the guests are going to have. Here are the categories of drinks
* Soda
* House wine
* Mixed drink (top shelf)

Let's assume the following:

__Drinking Behavior A__

In a four hour party, there are about five 45 minute periods (that adds up to 3 3/4 hours)
* Each person has a drink on average every 45 minutes. So this is on average five drinks. Some may drink more, some may drink less.
* Let's do the number of drinks for adults as a gaussian with mean of 5 and sd of 0.6. 
* For adults, two of their drinks have a 70% chance of being a mixed drink, 20% wine, 10% soda
* For adults, the third drink has 65% wine, 30% soda, 5% mixed drink
* For adults, the other drinks have 70% wine, 30% soda
* For kids, number of sodas is a gaussian with mean of 5 and sd of 1

Here's what a Gaussian $\mu=5; \sigma=0.6$ looks like (with integers; note that I've centered the bins on the integers)

In [8]:
gr()
niceHistBins(low, high) = low-0.5:high+0.5
niceHistBins(t::Tuple) = niceHistBins(t[1], t[2])
@> round.(Int, rand(Normal(5, 0.6),10000))  histogram(nbin=niceHistBins(2, 8), normalize=true, legend=nothing, line=nothing)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 2 
 
 
 3 
 
 
 4 
 
 
 5 
 
 
 6 
 
 
 7 
 
 
 8 
 
 
 0.0 
 
 
 0.1 
 
 
 0.2 
 
 
 0.3 
 
 
 0.4 
 
 
 0.5 
 
 
 0.6

Here's what a Gaussian with $\mu=5; \sigma=1$ looks likes (with integers)

In [9]:
@> round.(Int, rand(Normal(5, 1),10000))  histogram(nbin=niceHistBins(1, 9), normalize=true, legend=nothing, line=nothing)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 2 
 
 
 4 
 
 
 6 
 
 
 8 
 
 
 0.0 
 
 
 0.1 
 
 
 0.2 
 
 
 0.3

In [10]:
@everywhere function drinksForProb(vs::Array{Float64,1}, probMixed::Float64, probWine::Float64, probSoda::Float64)
    """
    Given a flat [0,1] random number array vs, determine if the drink is a mixed drink, wine, or soda
    according to the probabilities.
    Returns [nMixed, nWine, nSoda, 0]  # We always return soda in the third element 
    """
    @assert probMixed + probWine + probSoda ≈ 1.0
    
    nDrinks = zeros(Int, 4)
    
    nDrinks[1] += @> vs .<= probMixed sum                          # Mixed drinks
    nDrinks[2] += @> probMixed .< vs .<= probWine+probMixed sum    # Wine
    nDrinks[3] += @> probWine+probMixed .< vs sum                  # Sodas
    
    return nDrinks
end

In [11]:
# Some tests to be sure I did this right
@test drinksForProb([0.5, 0.1, 0.95], 0.7, 0.2, 0.1) == [2, 0, 1, 0]
@test drinksForProb([0.5, 0.8, 0.95], 0.7, 0.2, 0.1) == [1, 1, 1, 0]
@test drinksForProb([0.95, 0.95, 0.95], 0.7, 0.2, 0.1) == [0, 0, 3, 0]
@test drinksForProb([0.95, 0.95, 0.95], 0.8, 0.2, 0.0) == [0, 3, 0, 0]

Test Passed

In [12]:
@everywhere function drinkingBehaviorA(g::Array{Int,1})
    """
    Implement Drinking Behavior A (see above)
    g has the form [# of adults, # of teens, # of kids]
    """
    
    nDrinks = zeros(Int, 4)  # [nMixed, nWine, nSoda_adults, nSoda_kids]
    
    # Adult drinks
    nAdultDrinks = round.(Int, rand(Normal(5,0.6), g[1]) )
    nDrinks += @> rand(@> nAdultDrinks .>= 1 sum) drinksForProb(0.7, 0.2, 0.1)         # Drink 1
    nDrinks += @> rand(@> nAdultDrinks .>= 2 sum) drinksForProb(0.7, 0.2, 0.1)         # Drink 2
    nDrinks += @> rand(@> nAdultDrinks .>= 3 sum) drinksForProb(0.05, 0.65, 0.3)       # Drink 3
    for i in 4:maximum(nAdultDrinks)                                                   # Drinks 4, ...
        nDrinks += @> rand(@> nAdultDrinks .>= i sum) drinksForProb(0.00, 0.70, 0.30)
    end
    
    # Kids drinks (# of sodas)
    nDrinks[4] += @> round.(Int, rand(Normal(5, 1), g[2] + g[3]) ) sum
    
    nDrinks
end

### Define costs

With the number of people and the number of drinks, we can try several different cost plans for the party.  Below are functions that create data frame entries for each "party". 

In [13]:
@everywhere begin

    function makeCommonEntry(g::Array{Int,1}, d::Array{Int,1})
        """
           Make the common entries
               w = worker thread ID,   nA = #Adults,       nT = #Teens, nC = #Children
              nM = #Mixed drinks,      nW = #Wine drinks, 
             nSA = #sodas for adults, nSK = #sodas for kinds (teens+children)
        """
        DataFrame(w=myid(), nA=g[1], nT=g[2], nC=g[3], nM=d[1], nW=d[2], nSA=d[3], nSK=d[4])
    end

    function wtEntry(g::Array{Int,1}, d::Array{Int,1})
        """
           Costing for the WT Banquet Hall
           Wfood   = Cost for food,          WdrinkT = drink cost on tab plan,     wDrinkP = drink costs on package plan
           WtotalT = total cost on tab plan, WtotalP = total cost on package plan,    WT_P = total tab - total package
        """

        wtFood(g) = 23.0(g[1]+g[2]) + 13.0g[3]                   #  Food: $23/adult, $13/child
        wtDrinkTab(g, d) = 9.0d[1] + 7.5d[2] + 3d[3] + 3d[4]     #  Drink tab: Mixed=$9, Wine=$7.50, Soda=$3
        wtDrinkPkg(g, d) = 20.0(g[1] + g[2] + g[3])              #  Drink package: $20/person

        food = wtFood(g)
        drinkT = wtDrinkTab(g, d)
        drinkP = wtDrinkPkg(g, d)

        totalT = (1.0 + 0.2 + 0.0975)*(food + drinkT)                      # Tax is 9.75% and service is 20%
        totalP = (1.0 + 0.2 + 0.0975)*(food + drinkP)

        DataFrame(Wfood=food, WdrinkT=drinkT, WdrinkP=drinkP, WtotalT=totalT, WtotalP=totalP, WT_P=totalT-totalP)
    end

    function cprEntry(g::Array{Int,1}, d::Array{Int,1})
        """
           Costing for the cpr Banquet Hall
           Cfood   = Cost for food,          CdrinkT = drink cost on tab plan,     CDrinkP = drink costs on package plan
           CtotalT = total cost on tab plan, CtotalP = total cost on package plan,    CT_P = total tab - total package
        """

        cprFood(g) = 27.0*(g[1] + g[2]) + 15.0*g[3]                        # Food is $27/adult, $15/child
        cprDrinkTab(g, d) = 9.0*d[1] + 7.0*d[2] + 1.0*d[3] + 1.0*d[4]      # Drink tab: Mixed=$9, Wine=$7, Soda=$1
        cprDrinkPkg(g, d) = 3.0*(g[1]+g[2]+g[3]) + 9.0*d[1] + 7.0*d[2]     # Unlimited soda service $3/person 

        food = cprFood(g)
        drinkT = cprDrinkTab(g, d)
        drinkP = cprDrinkPkg(g, d)

        totalT = (1.0 + 0.2 + 0.1075)*(food + drinkT) + 500.0
        totalP = (1.0 + 0.2 + 0.1075)*(food + drinkP) + 500.0

        DataFrame(Cfood=food, CdrinkT=drinkT, CdrinkP=drinkP, CtotalT=totalT, CtotalP=totalP, CT_P=totalT-totalP)
    end
    
end

I have a spreadsheet with some pre-worked out examples. Let's check them. 

In [14]:
wtEntry([50, 25, 15], [50, 150, 100, 30])

,Wfood,WdrinkT,WdrinkP,WtotalT,WtotalP,WT_P
1,1920.0,1965.0,1800.0,5040.79,4826.7,214.087


In [15]:
cprEntry([50, 25, 15], [50, 150, 100, 30])

,Cfood,CdrinkT,CdrinkP,CtotalT,CtotalP,CT_P
1,2250.0,1630.0,1770.0,5573.1,5756.15,-183.05


## Make Parties

Time to party!

In [16]:
@everywhere function makeParty()
    """Make one party"""
    
    # Determine guests
    g = randomsInRanges(adultRange, teenRange, childRange)
    
    # Determine drinks
    d = drinkingBehaviorA(g)
    
    # Common entry
    c = makeCommonEntry(g, d)
    
    # Cost at WT
    c = hcat(c, wtEntry(g, d))
        
    # Cost at CPR
    c = hcat(c, cprEntry(g, d))
    
    c
    
end 

In [18]:
@time makeParty()

  0.000236 seconds (497 allocations: 137.141 KiB)


,w,nA,nT,nC,nM,nW,nSA,nSK,Wfood,WdrinkT,WdrinkP,WtotalT,WtotalP,WT_P,Cfood,CdrinkT,CdrinkP,CtotalT,CtotalP,CT_P
1,1,44,20,15,66,93,60,180,1667.0,2011.5,1580.0,4772.85,4212.98,559.871,1953.0,1485.0,1482.0,4995.18,4991.26,3.9225


**Choose from three options for how to proceed**

### Run single threaded

In [19]:
x = makeParty()
@time for i in 1:125000-1
    append!(x, makeParty())
end

 28.766436 seconds (67.67 M allocations: 17.288 GiB, 13.80% gc time)


### Restore from file

In [27]:
using JLD
function dataFromCache(filename)
    if isfile(filename)
        return JLD.load(filename)["x"]
    end
    return nothing
end

dataFromCache (generic function with 1 method)

In [29]:
x = dataFromCache("party1M.jld");

### Run in Parallel

In [20]:
@time x = @parallel (append!) for i = 1:nworkers()
    y = makeParty()
    for i in 1:125000-1
        append!(y, makeParty())
    end
    y
end ;

 60.550336 seconds (263.07 k allocations: 321.848 MiB, 0.79% gc time)


In [22]:
using JLD
@save "party1M.jld" x

## Results

The result is a [DataFrame](http://juliadata.github.io/DataFrames.jl/stable/) with information about each party. Let's just look at the top few rows.

In [23]:
head(x)

,w,nA,nT,nC,nM,nW,nSA,nSK,Wfood,WdrinkT,WdrinkP,WtotalT,WtotalP,WT_P,Cfood,CdrinkT,CdrinkP,CtotalT,CtotalP,CT_P
1,2,54,21,11,78,145,53,159,1868.0,2425.5,1720.0,5570.82,4655.43,915.386,2190.0,1929.0,1975.0,5885.59,5945.74,-60.145
2,2,44,19,11,63,109,51,152,1592.0,1993.5,1480.0,4652.19,3985.92,666.266,1866.0,1533.0,1552.0,4944.19,4969.03,-24.8425
3,2,44,19,17,73,109,37,181,1670.0,2128.5,1600.0,4928.55,4242.82,685.729,1956.0,1638.0,1660.0,5199.15,5227.92,-28.765
4,2,44,18,18,60,116,56,189,1660.0,2145.0,1600.0,4936.99,4229.85,707.137,1944.0,1597.0,1592.0,5129.86,5123.32,6.5375
5,2,47,23,19,64,119,58,207,1857.0,2263.5,1780.0,5346.35,4719.01,627.341,2175.0,1674.0,1676.0,5532.57,5535.18,-2.615
6,2,47,22,16,69,113,59,184,1795.0,2197.5,1700.0,5180.27,4534.76,645.506,2103.0,1655.0,1667.0,5413.58,5429.27,-15.69


How many parties did we get?

In [24]:
using Formatting

@> nrow(x) format(commas=true) println

1,000,000


How many rows are duplications? If there are a lot, that could mean we've oversampled. 

In [25]:
# Look for duplicate rows. We need to remove the w column as that's the worker node ID
ndupl = @> x @select(:nA, :nT, :nC, :nM, :nW, :nSA, :nSK) nonunique sum()

849

In [26]:
@>> ndupl/nrow(x)*100 printfmtln("{1:.2f}%")

0.08%


I think this is ok.

### Check coverage

We need to look at coverage (did we cover a large sample of possibilities?). How many possible set of guests should we have (assuming no correlations)?

In [27]:
tdiff(t::Tuple{Int,Int}) = t[2]-t[1]+1
totalGuestSetsExpected = tdiff(adultRange) * tdiff(teenRange) * tdiff(childRange)

3720

How many sets of guests did we generate?

In [28]:
gs = @> x @select(:nA, :nT, :nC)          # Select out the columns involving guests
@> nrow(gs) format(commas=true) println

1,000,000


In [29]:
totalGuestSets = @> gs unique nrow    # Get the unique set (remove all duplicates)

3720

In [31]:
# They should agree with what we expect
@test totalGuestSetsExpected == totalGuestSets

Test Passed

We now want to see how many samples there are for each guest set. To do this, we'll make a unique hash for each row of guest sets and see how often each appears in the full set of parties. 

In [32]:
gsHashes = [ hash(r) for r in eachrow(gs) ]
nHashes = @> gsHashes unique length

3720

In [33]:
@test nHashes == totalGuestSets # This should match the total number of guest sets

Test Passed

In [34]:
using FreqTables

In [35]:
gsFreq = freqtable(gsHashes) # Determine how often a hash (guest set) appears 
gsFreq[1:5]  # Just show the first five

5-element Named Array{Int64,1}
Dim1               │ 
───────────────────┼────
0x00023f9b13bb2633 │ 240
0x0012f8d42691d831 │ 239
0x00145e2cafb86bff │ 251
0x00321dc9a184eafe │ 290
0x003d384ad9002082 │ 276

In [36]:
histogram(gsFreq.array, title="Number of drink sets for guest set", xlab="Number of drink sets", legend=nothing)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 225 
 
 
 250 
 
 
 275 
 
 
 300 
 
 
 325 
 
 
 0 
 
 
 100 
 
 
 200 
 
 
 300 
 
 
 400 
 
 
 Number of drink sets for guest set 
 
 
 Number of drink sets

In [37]:
length(gsFreq)

3720

In [38]:
@> sum(gsFreq) format(commas=true) println  # Should be the number entries in the whole list

1,000,000


In [39]:
mean(gsFreq), minimum(gsFreq), maximum(gsFreq)

(268.81720430107526, 209, 328)

The [histogram](https://en.wikipedia.org/wiki/Histogram) above is a distribution of the number of drink sets per guest set (this is the distribution of the number of times a guest set appears in the list, presumably with different drinks). In order to cover a large number of drink situations, we want this distribution to go out far (going out too far leads to duplication and unnecessary parties). With 100,000 parties, the plot has a mean of 27. With 1M parties we get a mean of 268. The Gaussian distribution in the plot is expected from the [Central Limit Theorem](https://en.wikipedia.org/wiki/Central_limit_theorem) [it works!]. Note that this plot used to look stratified with three distinct peaks, and the distribution of guests had the edge bins at half the height as the rest. That was because I originally had the random number generation for the number of guests calling `round` instead of `floor`. Using `round` means that the values at the ends of the range get hit only half as often -- a good catch by this test. 

### Distributions of guests 
Who did we invite? Let's look at [histograms](https://en.wikipedia.org/wiki/Histogram) that display distributions of interesting results from the parties. For the number of guests, we generated flat distributions and that's what we should see.

In [41]:
h1 = @df x histogram( :nA, legend=false, nbins=niceHistBins(adultRange), title="#Adults")
h2 = @df x histogram( :nT, legend=false, nbins=niceHistBins(teenRange), title="#Teens")
h3 = @df x histogram( :nC, legend=false, nbins=niceHistBins(childRange), title="#Children")
plot(h1, h2, h3, layout=(3,1))

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 40 
 
 
 50 
 
 
 60 
 
 
 70 
 
 
 0 
 
 
 10000 
 
 
 20000 
 
 
 30000 
 
 
 #Adults 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 18 
 
 
 20 
 
 
 22 
 
 
 24 
 
 
 26 
 
 
 0 
 
 
 25000 
 
 
 50000 
 
 
 75000 
 
 
 100000 
 
 
 #Teens 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 12.5 
 
 
 15.0 
 
 
 17.5 
 
 
 20.0 
 
 
 22.5 
 
 
 0 
 
 
 20000 
 
 
 40000 
 
 
 60000 
 
 
 80000 
 
 
 #Children

Are there any correlations?

In [43]:
p1 = @df x histogram2d( :nA, :nT, nbin=15, xlab="Number of adults", ylab="Number of teens")
p2 = @df x histogram2d( :nA, :nC, nbin=15, xlab="Number of adults", ylab="Number of children")
p3 = @df x histogram2d( :nT, :nC, nbin=15, xlab="Number of teens", ylab="Number of children")
plot(p1, p2, p3, layout=3)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 40 
 
 
 50 
 
 
 60 
 
 
 70 
 
 
 18 
 
 
 20 
 
 
 22 
 
 
 24 
 
 
 26 
 
 
 28 
 
 
 Number of adults 
 
 
 Number of teens 
 
 
<image width="166" height="148" xlink:href="data:;base64,
iVBORw0KGgoAAAANSUhEUgAAAKYAAACUCAYAAAAZOlLRAAAEmklEQVR4nO3dzW4bVRyG8f85M45J
HDdJaSkpAaqoLRIIqVI/tgiWXAIrLoT7Ysumm9JdUZEQ9EOtk6YhNLVjx4ln5rDhAl5vohfp+a1f
nZPCI69GM6mqNkuI3h38qA1XhuqRkeuBtCvHf2gHLmby3dHf0O5OWdpV45F8dbcq3r1yST4z1q5J
s6T+tyydfPXqzvfS7vT1z9JO+y8OXDDChCXChCXChCXChCXChCXChCXChCXChCXChCXChCXChCXC
hCXChKV6mUebZKmSp2X8TNupj9It8ZhYmh1Iu9y10q5U+r87srbNgx35yDL+U9sNrku7NHkl3x3i
w5Nl87a04xcTlggTlggTlggTlggTlggTlggTlggTlggTlggTlggTlggTlggTlggTluqcV+Vx9f6F
tCuR5DOz+Ha2ZvuetEuTl/LdkWtpVi5/pe3mh/LV6exYO7M08pnRnGp3z4+0nfi4X0TE7OiRtKvW
P5N2/GLCEmHCEmHCEmHCEmHCEmHCEmHCEmHCEmHCEmHCEmHCEmHCEmHCEmHCUt202uNXERHt1a+l
XZru62cO70q7avRQO/DTb+W7S2hvuuum4sdLO/0Rtd7eU2149YF85qInfjB27WNtN9AfX6z3f9XO
5LE3/J8RJiwRJiwRJiwRJiwRJiwRJiwRJiwRJiwRJiwRJiwRJiwRJiwRJizVOfX1tfiWsGjPlvgT
xK9jim9ma1vt7XEREWn8lzYc7ooniv+WiDi/+Z02nC7z9rqeNjv6XdqVFe0xuoiIdkP8GOvbx9KM
X0xYIkxYIkxYIkxYIkxYIkxYIkxYIkxYIkxYIkxYIkxYIkxYIkxYSjkP5EdixpOfpF03f6v/AfN/
tGGlPgWlP+FT+lvSLs0OtPNWP5Lvjm4hzfLJnn7kpc+1ofgUUho/l+/ubX+jnSl+Z5RfTFgiTFgi
TFgiTFgiTFgiTFgiTFgiTFgiTFgiTFgiTFgiTFgiTFgiTFjS3lT1n27xXtol9eVbERGbX0iz/OaR
tGvVR78iIk3FR8qqFW12+Jt8d7t9X9qV6lA+M52L/39mf2u7Jb6Nudj7RTxTfNxPvhm4QIQJS4QJ
S4QJS4QJS4QJS4QJS4QJS4QJS4QJS4QJS4QJS4QJS4QJSymill+PNhn9IO3K8Ib+B8z2tTN769Iu
f3BNvltVjp+Kw26JQ7VtvnJXPrJtTqSd+g3NtPWlfHcaPdR2jfadUX4xYYkwYYkwYYkwYYkwYYkw
YYkwYYkwYYkwYYkwYYkwYYkwYYkwYYkwYalOS7SZFqfa7vCJfGbpD6RdtXFb2nXi28QiIsp0JO3q
D+9Iu+bsnXx3LMbSrJ1rH1iN0D/GGu25fKaq62uPJcbaZWnGLyYsESYsESYsESYsESYsESYsESYs
ESYsESYsESYsESYsESYsESYsESYs1XW9JY+L+rHP+UT/C860bbNxSzuv6B/mTIPr2t0nL6Rddazt
IiLa4SfSrre+K58Zr7UPwS52Hki7ssRjfLmZS7t0PtXOk28GLhBhwhJhwhJhwhJhwhJhwhJhwhJh
whJhwhJhwhJhwhJhwhJhwtK/3s3itBjo0tkAAAAASUVORK5CYII=
" transform="translate(55, 12)"/>
 
 
 
 
 
 
 
<image width="18" height="148" xlink:href="data:;base64,
iVBORw0KGgoAAAANSUhEUgAAABIAAACUCAYAAACeJ3mSAAABW0lEQVRogd3a23EDQQhE0V57nH8S
ztEvBpzE+aCkAG7RFxitdvRUf07A5/SU4OT0/UagRqBZF20UKAzUyFHqC4FUtOf+GFD6GtBTvwh0
EWhhtPpTIOaoDWifozCQc8TOo23RUmjXcpkjVdE+R0ZRDnqGyIlRlBOjKGfqQaD7ZkDpdRXVuwHN
qGiFos1V0RY6UhWpOWom260IqqiVo2kme1vXnCO1Ii6a6tplK+IcqTlSjli0y3ZtH4i1n012o8l2
XdvnaJhs5ohwYEXMEWv/XXceQUeEk9Nsstd1beEcsZ8QBJOcUg/s+xzB9hMOjFZsjlw0BHLRCCY5
pV6NwRUhHPkQ8bog6MiQFjqCxwhyVEGgYRWtc3TRS//TSnYNqohFKwUaVtGDLqEK3UItjHZVtAq6
px10L+a6dlW0ZiC1/e5fYy8cbVg0doygFdnnKIOijZqjQcfIk3yQL/9/fmk1aCz3ydQAAAAASUVO
RK5CYII=
" transform="translate(232, 12)"/>
 
 
 3500 
 
 
 4000 
 
 
 4500 
 
 
 5000 
 
 
 5500 
 
 
 6000 
 
 
 6500 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 40 
 
 
 50 
 
 
 60 
 
 
 70 
 
 
 12.5 
 
 
 15.0 
 
 
 17.5 
 
 
 20.0 
 
 
 22.5 
 
 
 Number of adults 
 
 
 Number of children 
 
 
<image width="166" height="148" xlink:href="data:;base64,
iVBORw0KGgoAAAANSUhEUgAAAKYAAACUCAYAAAAZOlLRAAAFWElEQVR4nO3dT29UZRiG8ec9553p
TEs7tKXQAlKJEKJxQ4z/FiqauHDvwo2fwMRP5Kdw6YqFMRrRmGCUCJEYimNL6bSFTunMmfP6Fe5u
yL24fusn7zMlF2dzZs5JVeqXEB0Mv5TmSt1Vj4x0sq/NTcba7t5ZeXcMXtN27/wqzeX9oby6Wbks
zbULG/KZaXIozeWd+9Jcs3JF3z09lubawaY0V8mbgZeIMGGJMGGJMGGJMGGJMGGJMGGJMGGJMGGJ
MGGJMGGJMGEp97uX5GH5yxl5QT6z3vlDG0za/6Fm+bq8O05G2uoLb0tz7clteXXbP6ftbrQvR0RE
xFT7osv06kfSXB7+Iq9uVsV/947WBldMWCJMWCJMWCJMWCJMWCJMWCJMWCJMWCJMWCJMWCJMWMol
5OcdRH5yT5qbLb8qnzkT77GWPCfNVV39gQdtmWqD23eksdScyLsj97W58Y58ZFl5Q5pLe9r3E8rm
p/LuGP8rjeXHP0pzXDFhiTBhiTBhiTBhiTBhiTBhiTBhiTBhiTBhiTBhiTBhKVXVonyzfH/0tTbY
aL9vjohIc6va4Ei7v9tdvyXvbh5+qw2mJI21G+/Ku6vhT9qZZ87LZ5ZK+91/ffCPtntuUd6d1m5q
Z06fS3NcMWGJMGGJMGGJMGGJMGGJMGGJMGGJMGGJMGGJMGGJMGGJMGEp5+qMPJyOtR/fl/kN+cxq
+2dpru2qn1N/gEPpag8dqC9qD9OfjX6Xd7dLF6W5/vkP5DPHT36Q5srlD7UDD/+Wd7dHW9JcerEn
zXHFhCXChCXChCXChCXChCXChCXChCXChCXChCXChCXChKWU62X55vLe3lfSXL2r3zMudUeaa89o
99/zzp/y7ubCm9pgpX3GKPp9+nT0WJubNfqZJ9rDBNr5ZWmufr4r7243P5bmysEDaY4rJiwR

There are no correlations among the guests. That's good, as we didn't put any in.

### Distributions of drinks

What did everyone drink?

In [46]:
@df x histogram( [:nM, :nW, :nSA, :nSK], layout=4, legend=false, nbins=15, title=["Mixed Drinks" "Wine" "Soda Adults" "Soda Kids"], 
    title_location=:center, line=nothing,
    color=[:green :darkgreen :darkblue :lightblue])

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 40 
 
 
 60 
 
 
 80 
 
 
 100 
 
 
 120 
 
 
 0 
 
 
 50000 
 
 
 100000 
 
 
 150000 
 
 
 200000 
 
 
 Mixed Drinks 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 90 
 
 
 120 
 
 
 150 
 
 
 180 
 
 
 210 
 
 
 0 
 
 
 20000 
 
 
 40000 
 
 
 60000 
 
 
 80000 
 
 
 100000 
 
 
 120000 
 
 
 Wine 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 25 
 
 
 50 
 
 
 75 
 
 
 100 
 
 
 0 
 
 
 50000 
 
 
 100000 
 
 
 150000 
 
 
 200000 
 
 
 250000 
 
 
 Soda Adults 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 125 
 
 
 150 
 
 
 175 
 
 
 200 
 
 
 225 
 
 
 250 
 
 
 275 
 
 
 0 
 
 
 50000 
 
 
 100000 
 
 
 150000 
 
 
 Soda Kids

Easier to think about number of drinks per person (a kid is a teen or child)

In [48]:
@df x histogram( [:nM./:nA :nW./:nA :nSA./:nA :nSK./(:nT.+:nC) ], line=nothing,
                color=[:green :darkgreen :darkblue :lightblue],
                layout=4, nbins=20, label="", title=["mixed drinks/adult" "wine/adult" "soda/adult" "soda/kid"] )

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1.0 
 
 
 1.2 
 
 
 1.4 
 
 
 1.6 
 
 
 1.8 
 
 
 0 
 
 
 50000 
 
 
 100000 
 
 
 150000 
 
 
 200000 
 
 
 mixed drinks/adult 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1.8 
 
 
 2.1 
 
 
 2.4 
 
 
 2.7 
 
 
 3.0 
 
 
 0 
 
 
 50000 
 
 
 100000 
 
 
 150000 
 
 
 200000 
 
 
 250000 
 
 
 wine/adult 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.6 
 
 
 0.8 
 
 
 1.0 
 
 
 1.2 
 
 
 1.4 
 
 
 1.6 
 
 
 1.8 
 
 
 0 
 
 
 100000 
 
 
 200000 
 
 
 300000 
 
 
 soda/adult 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 4.25 
 
 
 4.50 
 
 
 4.75 
 
 
 5.00 
 
 
 5.25 
 
 
 5.50 
 
 
 5.75 
 
 
 0 
 
 
 50000 
 
 
 100000 
 
 
 150000 
 
 
 200000 
 
 
 250000 
 
 
 soda/kid

### Cost results

How do the total costs compare? Let's try a [Box plot](https://en.wikipedia.org/wiki/Box_plot)

In [49]:
@df x boxplot( ["cpr Package" "cpr Tab" "WT Package" "WT Tab"], [ :CtotalP :CtotalT :WtotalP :WtotalT], 
               ylab="Cost \$", ylim=(3000, 8000), legend=false )

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 cpr Package 
 
 
 cpr Tab 
 
 
 WT Package 
 
 
 WT Tab 
 
 
 3000 
 
 
 4000 
 
 
 5000 
 
 
 6000 
 
 
 7000 
 
 
 8000 
 
 
 Cost $

Let's look at the distributions too...

In [50]:
histogram( [x[:CtotalP], x[:CtotalT], x[:WtotalP], x[:WtotalT]], bins=40, 
            layout=grid(4,1), color=[:red :blue :green :yellow], xlim=(3500, 8000), line=nothing,
            labels=["cpr Package" "cpr Tab" "WT Package" "WT Tab"])  # Note no commas in colors

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 4000 
 
 
 5000 
 
 
 6000 
 
 
 7000 
 
 
 8000 
 
 
 0 
 
 
 10000 
 
 
 20000 
 
 
 30000 
 
 
 40000 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 cpr Package 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 4000 
 
 
 5000 
 
 
 6000 
 
 
 7000 
 
 
 8000 
 
 
 0 
 
 
 10000 
 
 
 20000 
 
 
 30000 
 
 
 40000 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 cpr Tab 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 4000 
 
 
 5000 
 
 
 6000 
 
 
 7000 
 
 
 8000 
 
 
 0 
 
 
 10000 
 
 
 20000 
 
 
 30000 
 
 
 40000 
 
 
 50000 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 WT Package 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 4000 
 
 
 5000 
 
 
 6000 
 
 
 7000 
 
 
 8000 
 
 
 0 
 
 
 10000 
 
 
 20000 
 
 
 30000 
 
 
 40000 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 WT Tab

Let's compare the drink packages to paying by drink (tab)...

In [52]:
h1 = @df x histogram( :CT_P, nbin=40, legend=false, title="cpr: How much more Tab is compared to Package", line=nothing)
h1 = vline!([0])
h2 = @df x histogram( :WT_P, nbin=40, legend=false, title="WT: How much more Tab is compared to Package", 
                                      xlab="Price difference \$", line=nothing)
plot(h1, h2, layout=(2,1))

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -150 
 
 
 -100 
 
 
 -50 
 
 
 0 
 
 
 50 
 
 
 0 
 
 
 10000 
 
 
 20000 
 
 
 30000 
 
 
 40000 
 
 
 50000 
 
 
 60000 
 
 
 cpr: How much more Tab is compared to Package 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 250 
 
 
 500 
 
 
 750 
 
 
 1000 
 
 
 1250 
 
 
 0 
 
 
 20000 
 
 
 40000 
 
 
 60000 
 
 
 80000 
 
 
 WT: How much more Tab is compared to Package 
 
 
 Price difference $

Positive means that the price of putting beverages on a tab is *more* than the cost of the package. So using a tab is bad in those cases.

In the above, we see that for WT, doing drinks by tab is always much more expensive than the package. Don't do the tab at WT; the package is a better deal by far.

### Explore cpr where by package is favored

For cpr, the tab price is usually cheaper than the package price, and so the by drink (tab pricing) is the better option.

There are some parties where the opposite is true ... the package price beats the tab price (the positive part of the plot). What fraction of parties have this situation?

In [54]:
ctm = @where x (:CT_P .> 0);   # ctm = "cpr tab more"
percPkgBest = @>> nrow(ctm) / nrow(x) *100  printfmtln("{1:.1f}%")

9.2%


So for a little more than 9% of the simulated parties, the tab cost is more expensive than the package cost. What's special about those parties?

In [55]:
@df ctm histogram( [ :nA, :nT, :nC ], layout=(1,3), legend=false, nbins=15, title=["#Adults" "#Teens" "#Children"], 
    title_location=:left, line=nothing,
    color=[:red :darkred :pink])

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 40 
 
 
 50 
 
 
 60 
 
 
 70 
 
 
 0 
 
 
 5000 
 
 
 10000 
 
 
 15000 
 
 
 20000 
 
 
 25000 
 
 
 #Adults 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 20.0 
 
 
 22.5 
 
 
 25.0 
 
 
 27.5 
 
 
 0 
 
 
 5000 
 
 
 10000 
 
 
 15000 
 
 
 #Teens 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 12.5 
 
 
 15.0 
 
 
 17.5 
 
 
 20.0 
 
 
 22.5 
 
 
 0 
 
 
 5000 
 
 
 10000 
 
 
 15000 
 
 
 #Children

Hmm. These seem to be parties where there are few adults and lots of kids. Let's look

In [56]:
p1 = @df ctm histogram2d( :nA, :nT, nbin=15, xlab="Number of adults", ylab="Number of teens")
p2 = @df ctm histogram2d( :nA, :nC, nbin=15, xlab="Number of adults", ylab="Number of children")
p3 = @df ctm histogram2d( :nT, :nC, nbin=15, xlab="Number of teens",  ylab="Number of children")
plot(p1, p2, p3, layout=3)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 40 
 
 
 50 
 
 
 60 
 
 
 70 
 
 
 18 
 
 
 20 
 
 
 22 
 
 
 24 
 
 
 26 
 
 
 28 
 
 
 Number of adults 
 
 
 Number of teens 
 
 
<image width="166" height="148" xlink:href="data:;base64,
iVBORw0KGgoAAAANSUhEUgAAAKYAAACUCAYAAAAZOlLRAAAD9ElEQVR4nO3dTYpUVxyG8VOVKyiJ
koEgwZFzEbKIXocbyILcgyNHgQxcgJOIBmlB0f7QdFfb3amq/qh76/Y9GbiBt7CaetDnN/5zDkU9
3NHhnNHLra1aQg//epwNPv0jXbIcPHsQzT1/+Xs09+L4l3jvf2aLaO7teDuam/eH8d6L5SSaG4Yu
XrOWZTiY/eW1xGmsoB8lU+Nr2Fn6ZoYpJMMUkmEKyTCFZJhCMkwhGaaQDFNIhikkwxSSYQrJMIVk
mEJq/jvPj4ktXz2J5obXN+I19/fvR3OfLm5Gc5NFflRrMjqJ5hbDLJrr+mm891D7aK6WbO7r8CaP
s6WaaHO/mEIyTCEZppAMU0iGKSTDFJJhCskwhWSYQjJMIRmmkAxTSIYpJMMUUnNy8XM8fOPvP6O5
/e1H8Zpvju5Fc3sXTTS3253Fe0/Hx9Fc22fH3q5qG+9d6lU4N+RLbvQ4W3aLW8ovppAMU0iGKSTD
FJJhCskwhWSYQjJMIRmmkAxTSIYpJMMUkmEKyTCF1Jy22S1qpZTSvrsTze18ym5wK6WUg8v13uI2
H+XH3s6ujqK5rp9Hc3XIj73Vkh5n2+RRts3xiykkwxSSYQrJMIVkmEIyTCEZppAMU0iGKSTDFJJh
CskwhWSYQjJMITWnXf4g6WT7QTS3M/01XvPj+U/R3P4yO852Oj6M915eZWvW2mVzJbzBrZQVbnFb
5RK19IhcejNb9iDparPZ3n4xhWSYQjJMIRmmkAxTSIYpJMMUkmEKyTCFZJhCMkwhGaaQDFNIzefL
/HTR3r+/RXPv5/n7lDvny2jucHwQzc2Wn+O9uzWfLlpNdhhns1dqrfd9yFX4xRSSYQrJMIVkmEIy
TCEZppAMU0iGKSTDFJJhCskwhWSYQjJMIRmmkJqT/PnD8vbkbjT34Szv/bhm7zTO6iSa64b8Lcmh
Zj++pofP4ouyrsvmjqmtm19MIRmmkAxTSIYpJMMUkmEKyTCFZJhCMkwhGaaQDFNIhikkwxSSYQqp
OWrz9w93z29Fc5O2j9ecjLPb2dp+Fs0NQ36Or9bsprlS1n+cLb/F7fs5yrYKv5hCMkwhGaaQDFNI
hikkwxSSYQrJMIVkmEIyTCEZppAMU0iGKSTDFFLzZbiIhz+e3Y7mDuo0XnNej6K5rs9uhRuGRbx3
fJytXsdToz/mcbaUX0whGaaQDFNIhikkwxSSYQrJMIVkmEIyTCEZppAMU0iGKSTDFJJhCqmZjrJb
1Eop5bC9mc2N9+I1L9sv0Vz+IGl+e926xY+hllJKacLhVY7HXceam+EXU0iGKSTDFJJhCskwhWSY
QjJMIRmmkAxTSIYpJMMUkmEKyTCF9D/uzPSGqmQsTgAAAABJRU5ErkJggg==
" transform="translate(55, 12)"/>
 
 
 
 
 
 
 
<image width="18" height="148" xlink:href="data:;base64,
iVBORw0KGgoAAAANSUhEUgAAABIAAACUCAYAAACeJ3mSAAABW0lEQVRogd3a23EDQQhE0V57nH8S
ztEvBpzE+aCkAG7RFxitdvRUf07A5/SU4OT0/UagRqBZF20UKAzUyFHqC4FUtOf+GFD6GtBTvwh0
EWhhtPpTIOaoDWifozCQc8TOo23RUmjXcpkjVdE+R0ZRDnqGyIlRlBOjKGfqQaD7ZkDpdRXVuwHN
qGiFos1V0RY6UhWpOWom260IqqiVo2kme1vXnCO1Ii6a6tplK+IcqTlSjli0y3ZtH4i1n012o8l2
XdvnaJhs5ohwYEXMEWv/XXceQUeEk9Nsstd1beEcsZ8QBJOcUg/s+xzB9hMOjFZsjlw0BHLRCCY5
pV6NwRUhHPkQ8bog6MiQFjqCxwhyVEGgYRWtc3TRS//TSnYNqohFKwUaVtGDLqEK3UItjHZVtAq6
px10L+a6dlW0ZiC1/e5fYy8cbVg0doygFdnnKIOijZqjQcfIk3yQL/9/fmk1aCz3ydQAAAAASUVO
RK5CYII=
" transform="translate(232, 12)"/>
 
 
 500 
 
 
 1000 
 
 
 1500 
 
 
 2000 
 
 
 2500 
 
 
 3000 
 
 
 3500 
 
 
 4000 
 
<polyline clip-path="url(#clip3300)" style="stroke:#000000; stroke-width:1; stroke-opacity:1; fill:none" points="
 250.315,160.065 250.315,143.526 256.315,143.526 250.315,143.526 250.315,126.954 256.315,126.954 250.315,126.954 250.315,110.382 256.315,110.382 250.315,110.382 
 250.315,93.81 256.315,93.81 250.315,93.81 250.315,77.2378 256.315,77.2378 250.315,77.2378 250.315,60.6657 256.315,60.6657 250.315,60.6657 250.315,44.0936 
 256.315,44.0936 250.315,44.0936 250.315,27.5214 256.315,27.5214 250.315,27.5214 250.315,11.811 
 "/>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 40 
 
 
 50 
 
 
 60 
 
 
 70 
 
 
 12.5 
 
 
 15.0 
 
 
 17.5 
 
 
 20.0 
 
 
 22.5 
 
 
 Number of adults 
 
 
 Number of children 
 
 
<image width="166" height="148" xlink:href="data:;base64,
iVBORw0KGgoAAAANSUhEUgAAAKYAAACUCAYAAAAZOlLRAAAEOklEQVR4nO3dv24cVRiG8W/XAyn4
q0RIAURPR0fHJVBzHdwWBUVqKop0FBFKQIrAcbCN147Xjp312ruzc7gD9I4Uax/M86s/ndFIj6Y5
M2cmj7/5tlXo65+/ywYffZ8uWSc/PozmHj/5Kps7vh9f+9lFH809refR3Lx/GV/7en0czQ1tFa/Z
WnY/1YZsrOI03rrp1q4s/QvDFJJhCskwhWSYQjJMIRmmkAxTSIYpJMMUkmEKyTCF1E0m+fDNHz9E
cztP8xcPDvc/j+ZeXH4QzZ1cx5eugzaP5i6GWTTXbxbxtVvbhHPhixlV8csZtcWXM6r6qDifmEIy
TCEZppAMU0iGKSTDFJJhCskwhWSYQjJMIRmmkLr51Xvx8L1ff4rmXj3/Il7zr/mDaO5g+U40t3ed
b5bPp9mhA+vhKprrh2V87dbW6WS8Zjp7Ozvl2R54yiemkAxTSIYpJMMUkmEKyTCFZJhCMkwhGaaQ
DFNIhimk7mjxfjy82c2+mz7a/yxe88/LD6O5g6tsh3c2OY2vvdhks324Vz7qIP90x7rlO9v5ZLqv
3W3tA3SfmEIyTCEZppAMU0iGKSTDFJJhCskwhWSYQjJMIRmmkAxTSN35OjtIo

Indeed, these are cases where you have the minimum of adults and the maximum of kids. Let's look some more. Are we picking the parties that are skewed
to low drink consumption among adults and high consumption among kids? 

In [60]:
using StatsBase
function siHist(v1, v2, nbins1, nbins2, title)
    """
    Superimpose two histograms, one filled and one open
    """
    
    p = histogram( v1, nbins=nbins1, normalized=true, label="", line=nothing, title=title)
    
    f = fit(Histogram, v2, nbins=nbins2, closed=:left) ; f = normalize(f)
    p = plot!(f, seriestype=:step, normalized=true, label="")
end
    

siHist (generic function with 1 method)

Let's compare what these people drink (filled histogram) to the whole set (open histogram)

In [61]:
p1 = siHist( ctm[:nM], x[:nM], 15, 15, "Mixed Drinks")
p2 = siHist( ctm[:nW], x[:nW], 15, 15, "Wine")
p3 = siHist( ctm[:nSA], x[:nSA], 15, 18, "Soda Adults")
p4 = siHist( ctm[:nSK], x[:nSK], 15, 15, "Soda Kids")
plot(p1, p2, p3, p4, layout=4)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 40 
 
 
 60 
 
 
 80 
 
 
 100 
 
 
 120 
 
 
 0.00 
 
 
 0.01 
 
 
 0.02 
 
 
 0.03 
 
 
 0.04 
 
 
 0.05 
 
 
 0.06 
 
 
 Mixed Drinks 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 90 
 
 
 120 
 
 
 150 
 
 
 180 
 
 
 210 
 
 
 0.00 
 
 
 0.01 
 
 
 0.02 
 
 
 0.03 
 
 
 Wine 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
<polyline clip-path="url(#clip3503)" style="stroke:#e26f46; stroke-width:1; stroke-opacity:1; fill:none" points="
 340.317,175.813 340.317,175.79 356.317,175.79 356.317,174.549 372.317,174.549 372.317,162.107 388.316,162.107 388.316,137.665 404.316,137.665 404.316,126.882 
 420.316,126.882 420.316,126.137 436.316,126.137 436.316,126.034 452.316,126.034 452.316,126.078 468.316,126.078 468.316,126.856 484.316,126.856 484.316,133.039 
 500.316,133.039 500.316,150.204 516.315,150.204 516.315,167.801 532.315,167.801 532.315,174.755 548.315,174.755 548.315,175.756 564.315,175.756 564.315,175.813 
 580.315,175.813 580.315,175.813 
 "/>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 25 
 
 
 50 
 
 
 75 
 
 
 100 
 
 
 0.00 
 
 
 0.01 
 
 
 0.02 
 
 
 0.03 
 
 
 0.04 
 
 
 Soda Adults 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
<polyline clip-path="url(#clip3504)" style="stroke:#e26f46; stroke-width:1; stroke-opacity:1; fill:none" points="
 45.5535,375.813 45.5535,375.812 56.7897,375.812 56.7897,375.671 68.026,375.671 68.026,374.071 79.2622,374.071 79.2622,364.997 90.4985,364.997 90.4985,341.894 
 101.735,341.894 101.735,312.379 112.971,312.379 112.971,292.017 124.207,292.017 124.207,285.11 135.443,285.11 135.443,284.978 146.68,284.978 146.68,290.107 
 157.916,290.107 157.916,302.961 169.152,302.961 169.152,323.849 180.388,323.849 180.388,346.449 191.625,346.449 191.625,363.426 202.861,363.426 202.861,371.723 
 214.097,371.723 214.097,374.877 225.333,374.877 225.333,375.651 236.57,375.651 236.57,375.783 247.806,375.783 247.806,375.81 259.042,375.81 259.042,375.813 
 
 "/>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 125 
 
 
 150 
 
 
 175 
 
 
 200 
 
 
 225 
 
 
 250 
 
 
 275 
 
 
 0.000 
 
 
 0.005 
 
 
 0.010 
 
 
 0.015 
 
 
 0.020 
 
 
 Soda Kids 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
<polyline clip-path="url(#clip3505)" style="stroke:#e26f46; stroke-width:1; stroke-opacity:1; fill:none" points="
 342.107,375.813 342.107,375.795 356.64,375.795 356.64,374.512 371.174,374.512 371.174,363.522 385.708,363.522 385.708,341.92 400.242,341.92 400.242,319.565 
 414.775,319.565 414.775,296.405 429.309,296.405 429.309,275.816 443.843,275.816 443.843,266.409 458.377,266.409 458.377,275.348 472.91,275.348 472.91,295.545 
 487.444,295.545 487.444,317.771 501.978,317.771 501.978,340.639 516.512,340.639 516.512,361.236 531.045,361.236 531.045,373.027 545.579,373.027 545.579,375.64 
 560.113,375.64 560.113,375.813 574.647,375.813 574.647,375.813 
 "/>

Again, it's easier to look at drinks per person

In [62]:
p1 = siHist( ctm[:nM]./ctm[:nA], x[:nM]./x[:nA], 15, 15, "Mixed Drinks/Adult")
p2 = siHist( ctm[:nW]./ctm[:nA], x[:nW]./x[:nA], 15, 15, "Wine/Adult")
p3 = siHist( ctm[:nSA]./ctm[:nA], x[:nSA]./x[:nA], 15, 18, "Soda/Adult")
p4 = siHist( ctm[:nSK]./(ctm[:nT].+ctm[:nC]), x[:nSK]./(x[:nT].+x[:nC]), 15, 15, "Soda/Kid")
plot(p1, p2, p3, p4, layout=4)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1.00 
 
 
 1.25 
 
 
 1.50 
 
 
 1.75 
 
 
 2.00 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 4 
 
 
 Mixed Drinks/Adult 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1.8 
 
 
 2.1 
 
 
 2.4 
 
 
 2.7 
 
 
 3.0 
 
 
 0.0 
 
 
 0.5 
 
 
 1.0 
 
 
 1.5 
 
 
 2.0 
 
 
 2.5 
 
 
 Wine/Adult 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
<polyline clip-path="url(#clip3703)" style="stroke:#e26f46; stroke-width:1; stroke-opacity:1; fill:none" points="
 334.441,175.813 334.441,175.808 350.355,175.808 350.355,175.744 366.27,175.744 366.27,175.337 382.184,175.337 382.184,171.454 398.098,171.454 398.098,156.672 
 414.012,156.672 414.012,115.972 429.926,115.972 429.926,59.3312 445.84,59.3312 445.84,31.4961 461.754,31.4961 461.754,53.6909 477.669,53.6909 477.669,114.76 
 493.583,114.76 493.583,155.926 509.497,155.926 509.497,171.161 525.411,171.161 525.411,175.115 541.325,175.115 541.325,175.69 557.239,175.69 557.239,175.807 
 573.154,175.807 573.154,175.813 
 "/>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.6 
 
 
 0.8 
 
 
 1.0 
 
 
 1.2 
 
 
 1.4 
 
 
 1.6 
 
 
 1.8 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 Soda/Adult 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
<polyline clip-path="url(#clip3704)" style="stroke:#e26f46; stroke-width:1; stroke-opacity:1; fill:none" points="
 22.3377,375.813 22.3377,375.804 39.9218,375.804 39.9218,375.574 57.5058,375.574 57.5058,372.934 75.0899,372.934 75.0899,355.596 92.674,355.596 92.674,311.695 
 110.258,311.695 110.258,231.496 127.842,231.496 127.842,244.927 145.426,244.927 145.426,304.548 163.01,304.548 163.01,354.307 180.594,354.307 180.594,371.673 
 198.179,371.673 198.179,375.255 215.763,375.255 215.763,375.76 233.347,375.76 233.347,375.81 250.931,375.81 250.931,375.813 268.515,375.813 268.515,375.813 
 
 "/>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 4.0 
 
 
 4.5 
 
 
 5.0 
 
 
 5.5 
 
 
 6.0 
 
 
 0.0 
 
 
 0.5 
 
 
 1.0 
 
 
 1.5 
 
 
 2.0 
 
 
 2.5 
 
 
 Soda/Kid

So not only are there a lot of kids and few adults, the adults and the kids are drinking more soda. 

## Back Matter

In [63]:
versioninfo()

Julia Version 0.6.2
Commit d386e40c17 (2017-12-13 18:08 UTC)
Platform Info:
  OS: macOS (x86_64-apple-darwin14.5.0)
  CPU: Intel(R) Core(TM) i7-4980HQ CPU @ 2.80GHz
  WORD_SIZE: 64
  BLAS: libopenblas (USE64BITINT DYNAMIC_ARCH NO_AFFINITY Haswell)
  LAPACK: libopenblas64_
  LIBM: libopenlibm
  LLVM: libLLVM-3.9.1 (ORCJIT, haswell)


When rendered correctly, this [Jupyter Notebook](http://jupyter.org/) uses the [Fira Code font](https://github.com/tonsky/FiraCode); a monospaced font with programming [ligatures](https://en.wikipedia.org/wiki/Typographic_ligature).

In [2]:
# Set up the notebook style
function css_styling()
    styles = open("custom.css") do f
        readstring(f)
    end
end
display(HTML(css_styling()))

HTML{String}("<style>\n    .container { width:100% !important; }\n    @font-face {\n       font-family: 'Fira Code';\n               src: url(\"https://cdn.rawgit.com/dunovank/jupyter-themes/1e851888/jupyterthemes/fonts/monospace/firacode/firacode.otf\") format(\"opentype\");\n    }\n   .CodeMirror {\n      font-family: 'Fira Code';\n      font-variant-ligatures: initial;\n    }\n\n   .cm-string {\n       font-variant-ligatures: none;\n    }\n</style>\n<script>\n    MathJax.Hub.Config({\n            TeX: {\n                extensions: [\"AMSmath.js\"]\n                           },\n                tex2jax: {\n                    inlineMath: [ ['\$','\$'], [\"\\\\(\",\"\\\\)\"] ],\n                    displayMath: [ ['\$\$','\$\$'], [\"\\\\[\",\"\\\\]\"] ]\n                },\n                displayAlign: 'center', // Change this to 'center' to center equations.\n                \"HTML-CSS\": {\n                    styles: {'.MathJax_Display': {\"margin\": 4}}\n                }\n        });\n</script>")